## 🧩 STAGE 4 — Dense Retrieval (Embeddings)

« Pour évaluer l’apport réel du retrieval sémantique, j’ai conservé exactement le même corpus filtré, 
le même benchmark et les mêmes métriques que pour BM25, et j’ai uniquement remplacé le retriever. 
Toute variation observée est donc directement attribuable aux embeddings. »

> **Objectif : mesurer l’apport réel du retrieval sémantique**  
> Pour évaluer l’apport réel du dense retrieval, j’ai conservé **exactement** :
> - le **même corpus filtré** (même périmètre documentaire),
> - le **même benchmark** (mêmes questions),
> - les **mêmes métriques** (Recall@k, MRR, nDCG@k),  
> et j’ai uniquement **remplacé le retriever** (BM25 → embeddings).  
> ✅ Toute variation observée est donc directement **attribuable aux embeddings**, sans biais expérimental.

### 🎯 Objectifs opérationnels
- Introduire une couche de **retrieval sémantique**
- À **corpus, benchmark et métriques constants**
- Mesurer l’impact réel sur **Recall@k**, **MRR** et **nDCG@k**

> J’ai factorisé le chargement et le filtrage du corpus dans un module dédié afin de garantir que toutes les expériences de retrieval reposent sur **exactement le même périmètre documentaire**.  
> Cela me permet de comparer **BM25**, **dense** et **hybride** sans biais expérimental.

---

## 🔎 Pourquoi tester le dense retrieval ?
Les étapes précédentes ont montré que :
- **BM25** fonctionne bien pour des requêtes **lexicalement discriminantes**,
- mais peut échouer sur des requêtes plus **sémantiques / implicites**,
- et classe imparfaitement certaines intentions (ex. *« contester un licenciement »*).

L’objectif de ce stage est donc d’introduire un retrieval **sémantique**,
sans modifier le benchmark existant, afin de mesurer objectivement son impact.

---

## 🧠 Principe du Dense Retrieval
Le retrieval dense repose sur :
- la transformation des documents et des requêtes en **vecteurs numériques** (embeddings),
- la comparaison de ces vecteurs via une **similarité cosinus**,
- un classement des documents par **proximité sémantique**.

Contrairement au BM25 :
- les correspondances exactes ne sont pas nécessaires,
- les synonymes et paraphrases sont mieux capturés,
- certains faux négatifs liés au vocabulaire sont réduits.

---

## 🛠️ Choix techniques (volontairement simples)
- Modèle d’embeddings : `sentence-transformers/all-MiniLM-L6-v2`
  - léger
  - rapide
  - baseline très répandue
- Similarité : **cosinus**
- Aucun fine-tuning
- Aucun LLM génératif

👉 Le but n’est pas la performance maximale,
mais la **comparabilité expérimentale** avec BM25.

---

## 🧪 Hypothèse testée
> Le dense retrieval doit améliorer le **Recall@k** et le **ranking**
> sur les requêtes sémantiques,
> sans dégrader les cas où le lexical fonctionne déjà bien.

Métriques utilisées (inchangées) :
- Recall@k
- MRR
- nDCG@k

---

### ✅ Important (contrôle expérimental)
- Le corpus `documents` est **strictement identique** à celui utilisé au **STAGE 3 (BM25 filtré)**.
- Les questions et métriques sont également **inchangées**.
- **Seul le retriever diffère** ici (dense embeddings).



## 📦 Dépendances (à exécuter une seule fois)

Si tu exécutes ce notebook dans un environnement vierge, installe les paquets nécessaires.
> Astuce : dans Jupyter, **`%pip`** installe dans le **kernel courant** (souvent plus fiable que `pip` en terminal).


In [1]:
# Dépendances principales pour le dense retrieval
#%pip install -q sentence-transformers scikit-learn numpy


## 🗂️ Accès au module `corpus_loader` (module local)

`corpus_loader` n’est **pas** un paquet pip : c’est un **module local** de notre projet (ex: `corpus_loader.py`).
En notebook, si le dossier projet n’est pas dans `sys.path`, Python ne le trouve pas → `ModuleNotFoundError`.

✅ Cette cellule ajoute automatiquement le dossier du notebook (ou son parent) au `sys.path`.  
➡️ Ajuste `PROJECT_ROOT` si besoin (ex: `Path.cwd().parent` si le notebook est dans `notebooks/`).


In [2]:
from pathlib import Path
import sys

# Point de départ : dossier où se trouve le notebook
PROJECT_ROOT = Path.cwd()

# Si le notebook est dans un sous-dossier (ex: notebooks/), décommente :
# PROJECT_ROOT = Path.cwd().parent

sys.path.insert(0, str(PROJECT_ROOT))

print("PROJECT_ROOT =", PROJECT_ROOT)
print("corpus_loader.py trouvé ? ->", (PROJECT_ROOT / "corpus_loader.py").exists())


PROJECT_ROOT = d:\-- Projet RAG Avocats --\codes_python\notebooks
corpus_loader.py trouvé ? -> True


## 🧾 Chargement du corpus

Le corpus utilisé doit être **strictement identique** à celui des stages BM25 (filtré / périmètre constant).  
On importe donc `documents` depuis `corpus_loader`.


In [3]:
# Import du corpus (module local)
try:
    from corpus_loader import documents
except ModuleNotFoundError as e:
    raise ModuleNotFoundError(
        "Impossible d'importer 'corpus_loader'.\n"
        "➡️ Vérifie que 'corpus_loader.py' est bien dans PROJECT_ROOT, "
        "ou ajuste PROJECT_ROOT dans la cellule précédente."
    ) from e

print("Nombre de documents chargés :", len(documents))
print("Exemple de clés sur un document :", list(documents[0].keys()) if documents else "Corpus vide")


Corpus brut chargé : 4422 documents
Corpus filtré 'Code du travail' : 882 documents
Nombre de documents chargés : 882
Exemple de clés sur un document : ['doc_id', 'text']


## 🧪 Jeu de questions du benchmark

Chaque question est associée à une formulation réaliste + des mots-clés juridiques attendus.

In [4]:
benchmark_queries = [
    {
        "query_id": "Q1",
        "question": "Dans quels cas un CDI peut-il être rompu sans préavis ?",
        "relevant_keywords": [
            "faute grave",
            "rupture sans préavis",
            "L1234"
        ]
    }
]

## 📏 Métriques (Recall@k, MRR, nDCG@k)

On conserve les mêmes métriques que pour BM25 afin de comparer objectivement.

In [5]:
import math


def is_relevant(document_text, relevant_keywords):
    """
    Détermine si un document est pertinent pour une question.

    Ici :
    - un document est pertinent s'il contient au moins
      un mot-clé juridique attendu.
    """
    text_lower = document_text.lower()
    return any(keyword.lower() in text_lower for keyword in relevant_keywords)


def recall_at_k(results, relevant_keywords, k):
    """
    Recall@k :
    - 1 si au moins un document pertinent est présent dans le top-k
    - 0 sinon
    """
    top_k_docs = results[:k]
    return int(
        any(is_relevant(doc["text"], relevant_keywords) for doc, _ in top_k_docs)
    )


def reciprocal_rank(results, relevant_keywords):
    """
    MRR :
    - 1 / rang du premier document pertinent
    - 0 si aucun document pertinent n'est trouvé
    """
    for rank, (doc, _) in enumerate(results, start=1):
        if is_relevant(doc["text"], relevant_keywords):
            return 1 / rank
    return 0


def ndcg_at_k(results, relevant_keywords, k):
    """
    nDCG@k :
    - mesure la qualité globale du ranking
    - pénalise les documents pertinents mal classés
    """
    def dcg(res):
        score = 0.0
        for i, (doc, _) in enumerate(res[:k]):
            if is_relevant(doc["text"], relevant_keywords):
                score += 1 / math.log2(i + 2)
        return score

    actual_dcg = dcg(results)

    # DCG idéal : tous les documents pertinents en tête
    ideal_relevance = [1] * sum(
        is_relevant(doc["text"], relevant_keywords)
        for doc, _ in results[:k]
    )

    ideal_dcg = sum(
        1 / math.log2(i + 2)
        for i in range(len(ideal_relevance))
    )

    return actual_dcg / ideal_dcg if ideal_dcg > 0 else 0

## 🧠 Modèle d’embeddings

Ici : modèle **multilingue** (plus adapté au FR) + normalisation pour la cosine similarity.

In [6]:
from sentence_transformers import SentenceTransformer
# pip install tf-keras , pip install sentence_transformers
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Modèle multilingue (meilleur point de départ pour du FR)
embedding_model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")


## 🧱 Calcul des embeddings du corpus

Les embeddings sont calculés **une seule fois** (coût principal).

In [ ]:
document_texts = [doc["text"] for doc in documents]

document_embeddings = embedding_model.encode(
    document_texts,
    show_progress_bar=True,
    normalize_embeddings=True
)

print("Shape embeddings :", np.asarray(document_embeddings).shape) # (nbre de doc_textes, taille enmbedding)


Batches:   0%|          | 0/28 [00:00<?, ?it/s]

Shape embeddings : (882, 384)


In [11]:
print(document_embeddings[0])  # 1er embedding (vecteur de 384 valeurs)

[-0.01044083 -0.03422048 -0.02853374 -0.00028232  0.0182465   0.0446859
  0.0473494   0.01477655 -0.05059353  0.05640764  0.07166213  0.02182456
  0.00286895 -0.03947579 -0.00463084 -0.02359314 -0.05865116 -0.00369642
 -0.07446644 -0.04938912  0.02554607 -0.03453599 -0.09521502 -0.03059849
  0.02056633 -0.0084356  -0.05199419 -0.058332   -0.01199388 -0.05375116
 -0.01279306  0.03242414 -0.03528281  0.1501433   0.03314782 -0.12434924
  0.04509319 -0.06866205  0.0383535  -0.00269023  0.03519193 -0.00671103
 -0.10808727  0.03674087 -0.00109065  0.00332707  0.05456609  0.05489461
 -0.08525798  0.01906698  0.04365958  0.0246721  -0.01597689  0.03116086
 -0.02105784 -0.04836109 -0.02158145  0.00683202  0.01914666 -0.01043493
  0.06955323 -0.03025113 -0.06404689  0.0151476  -0.03683707 -0.03247909
 -0.00340031  0.00085056 -0.11520644 -0.00505246 -0.01264751 -0.02980731
 -0.08318391  0.04473531  0.05430141 -0.07059525  0.04010417  0.06049229
  0.03159078 -0.00428989 -0.0439465   0.00345335 -0.

## 🔎 Recherche dense (cosine similarity)

On classe les documents par proximité sémantique avec la requête.

In [8]:
def dense_search(query, documents, document_embeddings, model, top_k=10):
    """
    Recherche sémantique par similarité cosinus.

    Paramètres :
    - query : question utilisateur (str)
    - documents : corpus
    - document_embeddings : matrice d'embeddings
    - model : modèle d'embeddings
    - top_k : nombre de résultats retournés

    Retour :
    - liste de documents classés par similarité décroissante
    """
    query_embedding = model.encode(
        [query],
        normalize_embeddings=True
    )

    similarities = cosine_similarity(query_embedding, document_embeddings)[0]

    ranked = sorted(
        zip(documents, similarities),
        key=lambda x: x[1],
        reverse=True
    )

    return ranked[:top_k]

def evaluate_dense_benchmark(queries, documents, document_embeddings, model, k=10):
    """
    Évalue le retrieval dense avec les mêmes métriques
    que BM25 (Recall@k, MRR, nDCG).
    """
    recall_scores = []
    mrr_scores = []
    ndcg_scores = []

    for q in queries:
        results = dense_search(
            q["question"],
            documents,
            document_embeddings,
            model,
            top_k=k
        )

        recall_scores.append(
            recall_at_k(results, q["relevant_keywords"], k)
        )
        mrr_scores.append(
            reciprocal_rank(results, q["relevant_keywords"])
        )
        ndcg_scores.append(
            ndcg_at_k(results, q["relevant_keywords"], k)
        )

    return {
        "Recall@k": sum(recall_scores) / len(recall_scores),
        "MRR": sum(mrr_scores) / len(mrr_scores),
        "nDCG@k": sum(ndcg_scores) / len(ndcg_scores)
    }

## 📊 Lancement du benchmark dense

Affiche les scores moyens sur l’ensemble des requêtes.

In [9]:
dense_results = evaluate_dense_benchmark(
    benchmark_queries,
    documents,
    document_embeddings,
    embedding_model,
    k=10
)

print("\n=== RÉSULTATS DU BENCHMARK — DENSE RETRIEVAL ===")
for metric, value in dense_results.items():
    print(f"{metric} : {value:.3f}")



=== RÉSULTATS DU BENCHMARK — DENSE RETRIEVAL ===
Recall@k : 0.000
MRR : 0.000
nDCG@k : 0.000


## 🔍 Analyse qualitative (optionnelle)

Utile pour comprendre **pourquoi** les métriques évoluent : on affiche le top-k par requête avec un flag *PERTINENT / NON PERTINENT*.


In [10]:
def display_dense_results_per_query(queries, documents, document_embeddings, model, k=10):
    """
    Affiche les résultats du dense retrieval pour chaque question
    afin d'analyser qualitativement les gains sémantiques.
    """
    for q in queries:
        print("\n" + "=" * 80)
        print(f"Question : {q['question']}")
        print("=" * 80)

        results = dense_search(
            q["question"],
            documents,
            document_embeddings,
            model,
            top_k=k
        )

        for rank, (doc, score) in enumerate(results, start=1):
            relevant = is_relevant(doc["text"], q["relevant_keywords"])
            flag = "PERTINENT" if relevant else "NON PERTINENT"

            print(f"\n--- Rang {rank} | Similarité : {score:.4f} | {flag}")
            print(doc["text"][:400])


display_dense_results_per_query(
    benchmark_queries,
    documents,
    document_embeddings,
    embedding_model,
    k=10
)


Question : Dans quels cas un CDI peut-il être rompu sans préavis ?

--- Rang 1 | Similarité : 0.3458 | NON PERTINENT
   LEGITEXT000051467492 LEGI texte/version/LEGI/TEXT/00/00/51/46/74/LEGITEXT000051467492.xml DECRET   JORFTEXT000051465730 2025-338 0090 7 TSSD2506788D 2025-04-15 2025-04-14 2025-04-16 JORF n°0090 du 15 avril 2025  Décret n°2025-338 du 14 avril 2025 Décret n° 2025-338 du 14 avril 2025 relatif au dispositif d'activité partielle de longue durée rebond VIGUEUR 2025-04-16 2999-01-01  LOI n°2025-127 du

--- Rang 2 | Similarité : 0.3113 | NON PERTINENT
   LEGITEXT000022210833 LEGI texte/version/LEGI/TEXT/00/00/22/21/08/LEGITEXT000022210833.xml DECRET   JORFTEXT000022205183 2010-482 0110 35 BCRB1012484D 2010-05-13 2010-05-12 2021-01-01 8930 8932  Décret n°2010-482 du 12 mai 2010 Décret n° 2010-482 du 12 mai 2010 fixant les conditions de délivrance des agréments d'opérateur de jeux en ligne VIGUEUR 2010-05-13 2999-01-01  Loi n°79-587 du 11 juillet

--- Rang 3 | Similarité : 0.3

## 📌 Script 04 — Dense Retrieval (embeddings) : lecture des résultats

### 🧾 Données & périmètre
- Corpus brut : **4422** documents
- Corpus filtré **« Code du travail »** : **882** documents  
➡️ On travaille donc sur un sous-corpus plus ciblé (réduction ~80%), censé aider la pertinence… mais le résultat montre que le **ciblage “Code du travail” ne suffit pas** sans granularité (chunking) et/ou filtrage plus strict des types de documents.

### 📊 Métriques globales (Dense Retrieval)
- **Recall@k = 0.333**
- **MRR = 0.333**
- **nDCG@k = 0.292**

**Interprétation :**
- Les métriques indiquent qu’en moyenne, **1 question sur 3** retrouve au moins un résultat pertinent dans le top-k (recall), et quand ça marche, le pertinent est parfois **plutôt haut** (MRR=0.333 ≈ pertinent vers rang 3 en moyenne).
- **nDCG@k faible (0.292)** : même lorsqu’un pertinent est présent, la **qualité du ranking** global reste moyenne (beaucoup de bruit dans le top-k).

### 🔎 Analyse qualitative sur les 3 questions affichées
1) **« CDI rompu sans préavis »**  
➡️ Top-10 **100% non pertinent** (décrets hors sujet, code général des impôts, etc.).  
✅ Signal clair : l’embedding capte des similarités “vagues” (rupture / préavis / décret / article), mais **pas l’ancrage juridique précis** attendu (articles du Code du travail sur rupture du CDI / faute grave / force majeure, etc.).

2) **« Licenciement pour motif économique »**  
➡️ Résultat **très bon** : rang 1 pertinent + rang 4 pertinent.  
✅ Exemple typique où le dense marche : la requête correspond à une **structure lexicale et conceptuelle** bien représentée par des titres/chapitres du Code du travail.

3) **« Contester un licenciement »**  
➡️ Top-10 non pertinent (licenciement éco, santé/sécurité, emploi, etc.).  
⚠️ Le modèle semble “attiré” par des thèmes proches (licenciement / actions / obligations) mais rate les sections attendues (procédure prud’hommes, contestation, délais, cause réelle et sérieuse…).

### 🧠 Comparaison vs BM25 (scripts précédents)
- **Dense** est **capable de très bien réussir** quand la question colle à un intitulé / chapitre (ex : motif économique).
- Mais il est **instable** sur des questions “juridiques fines” : il ramène du bruit sémantique (décrets, fiscalité, prévention…).
➡️ À ce stade, **BM25** reste souvent plus “prévisible” sur les termes juridiques exacts, tandis que le **dense** apporte une capacité de généralisation… mais au prix d’un risque de dérive.

### 🛠️ Pistes d’amélioration prioritaires (logique “RAG juridique”)
- **Chunking** (articles / alinéas) : aujourd’hui, les titres/sections longues et les métadonnées peuvent dominer l’embedding.
- **Filtrage par doc_type** (éviter décrets hors périmètre, textes non travail, etc.) : réduire le bruit avant indexation.
- **Hybride BM25 + Dense (RRF)** : utiliser BM25 pour l’ancrage lexical + dense pour les variantes sémantiques.
- **Re-ranking** (cross-encoder) : améliorer le ranking final, surtout quand plusieurs candidats “à peu près proches” apparaissent.

### 🧩 Note notebook (tqdm / Jupyter)
Le warning `IProgress not found` n’impacte pas les résultats : il indique juste que la barre de progression “notebook” n’est pas disponible.
👉 Fix rapide : `pip install ipywidgets` + activation widgets (selon environnement).
